# Simulation experiment 

Run entire simulation experiment multiple times to generate confidence interval

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../../")
from functions import pipelines

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [2]:
# Parameters
dataset_name = "Human_analysis"
analysis_name = 'analysis_3'
NN_architecture = 'NN_2500_30'
file_prefix = "Partition"
num_simulated_experiments = 50
lst_num_partitions = [1, 2, 3, 5, 10, 20,
                    30, 50]
corrected = False
use_pca = True
num_PCs = 10

iterations = range(5) 
num_cores = 5

In [3]:
# Input
base_dir = os.path.abspath(
      os.path.join(
          os.getcwd(), "../.."))

normalized_data_file = os.path.join(
    base_dir,
    dataset_name,    
    "data",
    "input",
    "recount2_gene_normalized_data.tsv.xz")

experiment_ids_file = os.path.join(
    base_dir,
    dataset_name,
    "data",
    "metadata",
    "recount2_experiment_ids.txt")

In [4]:
# Output files
similarity_uncorrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "analysis_3_similarity_uncorrected.pickle")

ci_uncorrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "analysis_3_ci_uncorrected.pickle")

similarity_permuted_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "analysis_3_permuted")

In [5]:
# Run multiple simulations - uncorrected
results = Parallel(n_jobs=num_cores, verbose=100)(
    delayed(
        pipelines.matched_simulation_experiment_uncorrected)(i,
                                                             NN_architecture,
                                                             dataset_name,
                                                             analysis_name,
                                                             num_simulated_experiments,
                                                             lst_num_partitions,
                                                             corrected,
                                                             use_pca,
                                                             num_PCs,
                                                             file_prefix,
                                                             normalized_data_file,
                                                             experiment_ids_file) for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed: 61.1min
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 73.5min remaining: 110.3min
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed: 77.8min remaining: 51.8min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 88.9min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 88.9min finished


In [6]:
base_dir

'/home/alexandra/Documents/Repos/Batch_effects_simulation'

In [7]:
# permuted score
permuted_score = results[0][0]

In [8]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score
number of partitions,,,,,
1,0.999960,0.999985,0.999983,0.999961,0.999982
2,0.928445,0.956821,0.907806,0.911209,0.936189
3,0.846454,0.832273,0.847968,0.863628,0.841786
5,0.748486,0.738688,0.716731,0.719037,0.744306
10,0.628535,0.623793,0.616556,0.593496,0.611436
20,0.690121,0.764841,0.681735,0.714280,0.724823
30,0.708795,0.787579,0.674464,0.674068,0.722343
50,0.745905,0.791791,0.736716,0.811924,0.789731


In [9]:
# Get median for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

,score
number of partitions,
1,0.999974
2,0.928094
3,0.846421
5,0.733450
10,0.614763
20,0.715160
30,0.713450
50,0.775213


In [10]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

,score
number of partitions,
1,0.000004
2,0.006300
3,0.003606
5,0.004633
10,0.004295
20,0.010376
30,0.014720
50,0.010215


In [11]:
# Get confidence interval for each row (number of experiments)
err = std_scores*1.96

In [12]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

,ymin,ymax
number of partitions,,
1,0.999966,0.999982
2,0.915746,0.940442
3,0.839354,0.853489
5,0.724369,0.742530
10,0.606346,0.623181
20,0.694823,0.735497
30,0.684599,0.742300
50,0.755191,0.795236


In [13]:
mean_scores

,score
number of partitions,
1,0.999974
2,0.928094
3,0.846421
5,0.733450
10,0.614763
20,0.715160
30,0.713450
50,0.775213


In [14]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_uncorrected_file)
ci.to_pickle(ci_uncorrected_file)
np.save(similarity_permuted_file, permuted_score)